In [190]:
from bs4 import BeautifulSoup
import urllib3
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

http = urllib3.PoolManager()

urls = ["https://resumes.indeed.com/resume/19936d88d452cba8?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
        "https://resumes.indeed.com/resume/fc69b971ab0f7feb?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/e7a80d47fd3b870a?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/a0e2efc28ffe7076?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/4416a22a685fe75f?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/a7ac5303fd482fbb?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/01303e67a9a9f907?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/adf5c0dcf7fb32c1?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/0dcb97d5e222ff21?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/e8aaa1f939a0a899?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt"]


newDict = {}
newDict['Category'] = "Data Scientist"
newDict['Resume'] = []
counter = 0
for x in range(0,len(urls)):
    response = http.request('GET', urls[x])
    soup = BeautifulSoup(response.data,'html.parser',  from_encoding="utf-8")
    reqText = soup.find_all('script')[2].text
    newText  = re.findall(r"\'(.*?)\'",reqText)
    resumeText = GetRequiredResumeDetails(newText)
    str1 = ''.join(resumeText)
    
    newDict['Resume'].append(str1)
df= pd.DataFrame(newDict)
print (df)

         Category                                             Resume
0  Data Scientist  Skills * Programming Languages: Python (pandas...
1  Data Scientist  Education Details \nMay 2013 to May 2017 B.E  ...
2  Data Scientist  Areas of Interest Deep Learning, Control Syste...
3  Data Scientist  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4  Data Scientist  Education Details \n MCA   YMCAUST,  Faridabad...
5  Data Scientist  SKILLS C Basics, IOT, Python, MATLAB, Data Sci...
6  Data Scientist  Skills â¢ Python â¢ Tableau â¢ Data Visuali...
7  Data Scientist  Education Details \n B.Tech   Rayat and Bahra ...
8  Data Scientist  Personal Skills â¢ Ability to quickly grasp t...
9  Data Scientist  Expertise â Data and Quantitative Analysis â...


In [151]:
import json
from pprint import pprint

def GetRequiredResumeDetails(ScrapedText):
    value = newText[0]
    extractedJSONStr = value.encode('utf8').decode('unicode_escape')
    jsonObj = json.loads(extractedJSONStr)
    #pprint(jsonObj['resumeModel'])
    requiredTags = ['fullName','headline','education','additionalInformation','workExperience', 'headline','skills']
    educationRequiredString = []
    skillsRequiredString = []
    companyDetails = []
    counter = 0
    resumeText = []
    totalDataDict = {}
    for j in jsonObj['resumeModel']:
        if(j in requiredTags):
            if(j == 'additionalInformation'):
                reqText = re.sub(r'\s+',' ',jsonObj['resumeModel'][j])
                #print (reqText)
                resumeText.append(reqText)
            if(j == 'education'):
                for x in range(0,len(jsonObj['resumeModel'][j])):
                    reqStr = jsonObj['resumeModel'][j][x]['dateRange']+" "+jsonObj['resumeModel'][j][x]['degree']+" "+jsonObj['resumeModel'][j][x]['field']+" "+jsonObj['resumeModel'][j][x]['location']+" "+jsonObj['resumeModel'][j][x]['university']
                    educationRequiredString.append(reqStr)
                educationDetails = "Education Details \n"
                educationDetails += "\n".join(educationRequiredString)
                resumeText.append(educationDetails)
            if(j == 'fullName'):
                #print (jsonObj['resumeModel'][j])
                resumeText.append(jsonObj['resumeModel'][j])

            if(j == 'headline'):
                #print (jsonObj['resumeModel'][j])
                resumeText.append(jsonObj['resumeModel'][j])

            if(j == 'skills'):
                #print  (len(jsonObj['resumeModel'][j]))
                for a in range(0,len(jsonObj['resumeModel'][j])):
                    monthsOfExperience = ""
                    if(jsonObj['resumeModel'][j][a]['monthsOfExperience'] == 0):
                        monthsOfExperience = "Less than 1 year"
                    else:
                        monthsOfExperience = jsonObj['resumeModel'][j][a]['monthsOfExperience']
                    string = jsonObj['resumeModel'][j][a]['skill']+"- Exprience - "+str(monthsOfExperience)+" months"
                    skillsRequiredString.append(string)

                skillDetails = "Skill Details \n"
                skillDetails += "\n".join(skillsRequiredString)
                #print (skillDetails)
                resumeText.append(skillDetails)

            if(j == "workExperience"):
                for b in range(0,len(jsonObj['resumeModel'][j])):
                    for tag in ['company','description']:
                        wkString = tag+" - "+jsonObj['resumeModel'][j][b][tag]
                        companyDetails.append(wkString)

                compDetails = "Company Details \n"
                compDetails += "\n".join(companyDetails)
                #print (compDetails)
                resumeText.append(compDetails)
    return resumeText